## ewf-ext-02-03-07 - NDVI anomalies of growing season per parcel 

NDVI anomalies of growing season per parcel

---

### <a name="service">Service definition

In [ ]:
service = dict([('title', 'NDVI anomalies of growing season per parcel'),
                ('abstract', 'NDVI anomalies of growing season per parcel'),
                ('id', 'ewf-ext-02-03-07')])

### <a name="parameter">Parameter Definition 

In [ ]:
regionOfInterest = dict([('id', 'regionOfInterest'),
                         ('value', 'POLYGON ((-8.864205 38.88616500000001, -8.864205 38.986165, -8.964205000000002 38.986165, -8.964205000000002 38.88616500000001, -8.864205 38.88616500000001))'),
                         ('title', 'WKT Polygon for the Region of Interest'),
                         ('abstract', 'Set the value of WKT Polygon')])

nameOfRegion = dict([('id', 'nameOfRegion'),
                     ('value', 'P001'),
                     ('title', 'Name of Region'),
                     ('abstract', 'Name of the region of interest'),
                     ('minOccurs', '1')])

### <a name="runtime">Runtime parameter definition

**Input identifiers**

This is the NDVI stats' identifiers

In [ ]:
# 2015, 2015
#difNdvi
input_identifiers = ('LE07_ndviStats_P001_2015005_2015365.xlsx', 'LTA_LE07_ndviStats_P001_2015_2015.xlsx')

**Input references**

This is the MODIS stack catalogue references

In [ ]:
input_references = tuple(['https://catalog.terradue.com/modis/search?format=atom&uid={0}'.format(pid) for pid in input_identifiers])

**Data path**

This path defines where the data is staged-in. 

In [ ]:
data_path = ""

**Aux folders**

In [ ]:
output_folder = ''

#### Import Modules

In [ ]:
import os
import shutil

import sys
import string
import numpy as np
from osgeo import gdal, ogr, osr
from shapely.wkt import loads

import datetime

import pdb

import pandas as pd

#### Auxiliary vars

In [ ]:
check_results = True

#### Workflow

Load data (agg)

In [ ]:
# get file paths
file_list = [os.path.join(data_path, in_id.split('/')[-1]) for in_id in input_identifiers]


# load data into a python dictionary
# key -> variable name
# content -> list of pandas dataframe, one per season (TS)
data_agg = {}

var_names = ['start_growing_season', 'end_growing_season', 'dif_ndvi', 'cumulative_ndvi', 'peak_ndvi']

f = file_list[0]
for var in var_names:

    df = pd.read_excel (f, sheet_name=var)
    #print (df)
    
    # remove useless column
    df = df.drop(columns=['Unnamed: 0'])
    
    data_agg[var] = df

Load data (LTA)

In [ ]:
# get file paths
file_list = [os.path.join(data_path, in_id.split('/')[-1]) for in_id in input_identifiers]


# load data into a python dictionary
# key -> variable name
# content -> list of pandas dataframe, one per season (TS)
data_LTA = {}

var_names = ['start_growing_season', 'end_growing_season', 'dif_ndvi', 'cumulative_ndvi', 'peak_ndvi']

f = file_list[1]
for var in var_names:

    df = pd.read_excel (f, sheet_name=var)
    #print (df)
    
    # remove useless column
    df = df.drop(columns=['Unnamed: 0'])
    
    data_LTA[var] = df

Compute Amomalies

In [ ]:
# new python dictionary to store LTAs
anom_data = {}

anom_data = data_agg.copy()

# to each var computes mean
for var in var_names:

    anom_data[var][var] = data_LTA[var][var].div(data_agg[var][var])


#### write output

In [ ]:
name_parts = input_identifiers[0].split('.')[0].split('_')

#Anomaly_LE07_difNdvi_P001_2015005_2015020_LTA_2015-2015

mission = name_parts[0]
prod = name_parts[1]
aoi_name = name_parts[2]
start_date = name_parts[3]
end_date = name_parts[4]

name_parts = input_identifiers[1].split('.')[0].split('_')

start_year_LTA = name_parts[4]
end_year_LTA = name_parts[5]

#start_date = str(anom_data['start_growing_season']['start_date'][0].year)
#end_date = str(anom_data['start_growing_season']['end_date'][0].year)

excel_output_name = '_'.join(['Anomaly', mission, prod, aoi_name, start_date, end_date, 'LTA', start_year_LTA, end_year_LTA]) + '.xlsx'
    
excel_output_name = os.path.join(output_folder, excel_output_name)

print(excel_output_name)


with pd.ExcelWriter(excel_output_name) as writer:  # doctest: +SKIP
    
    for key in anom_data:
    
        anom_data[key].to_excel(writer, sheet_name=key)
